<a href="https://colab.research.google.com/github/jamessutton600613-png/GC/blob/main/Untitled187.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell ID: 78f97a35
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
# Removed FuncAnimation as we are generating static plots
# from matplotlib.animation import FuncAnimation

# --- Core GQR Logic (from the KIE/Temp comparative model) ---
# This class provides the unique physics parameters for the simulation
class GQR_Parameters:
    def __init__(self, temperature=290.0, substrate_isotope='light'):
        # --- GQR Model Parameters ---
        self.optimal_temp = 290.0
        self.coherence_width = 70.0
        self.base_energy_increment = 1.2
        self.mass_effect_on_increment = 0.5
        self.mass_factor = 1.0 if substrate_isotope == 'light' else 2.0

        # --- Calculate Key Probabilities ---
        # 1. Probability of a successful ratchet "click"
        quantum_coherence = np.exp(-((temperature - self.optimal_temp)**2) / (2 * self.coherence_width**2))
        self.p_confirm = quantum_coherence * 0.98

        # 2. Energy increment per successful click (less for heavy isotopes)
        self.energy_increment = self.base_energy_increment - (self.mass_effect_on_increment * self.mass_factor)

# --- GPU-based 3D Simulation ---

def initialize_3d_system(grid_size=64):
    """ Initializes the potential field and an empty grid for the wavefunction. """
    V = cp.zeros((grid_size, grid_size, grid_size), dtype=cp.float32)
    barrier_width = 4
    barrier_height = 15.0  # Slightly higher barrier
    barrier_start = grid_size // 2 - barrier_width // 2
    barrier_end = grid_size // 2 + barrier_width // 2
    V[barrier_start:barrier_end, :, :] = barrier_height
    p680_depth = -50.0
    V[barrier_end:, :, :] = p680_depth # Corrected typo from p660_depth

    return V

def create_wave_packet(grid_size, energy_level):
    """ Creates a wave packet with a specific energy (momentum). """
    psi = cp.zeros((grid_size, grid_size, grid_size), dtype=cp.complex64)
    center = cp.array([grid_size // 4, grid_size // 2, grid_size // 2])
    sigma = cp.array([5, 10, 10])
    x, y, z = cp.ogrid[:grid_size, :grid_size, :grid_size]

    # Energy level translates to initial momentum (kx)
    kx = cp.sqrt(2 * energy_level) if energy_level > 0 else 0

    gaussian = cp.exp(-((x - center[0])**2 / (2 * sigma[0]**2) +
                        (y - center[1])**2 / (2 * sigma[1]**2) +
                        (z - center[2])**2 / (2 * sigma[2]**2)))
    plane_wave = cp.exp(1j * kx * x)
    psi = gaussian * plane_wave
    psi /= cp.sqrt(cp.sum(cp.abs(psi)**2))
    return psi

def update_wavefunction_gpu(psi, V, dt=0.01):
    """ Performs one step of the FDTD simulation. """
    laplacian_psi = (cp.roll(psi, -1, axis=0) + cp.roll(psi, 1, axis=0) +
                     cp.roll(psi, -1, axis=1) + cp.roll(psi, 1, axis=1) +
                     cp.roll(psi, -1, axis=2) + cp.roll(psi, 1, axis=2) - 6 * psi)
    psi_new = psi - 1j * dt * (-0.5 * laplacian_psi + V * psi)
    return psi_new

# --- Main Simulation and Visualization ---
if __name__ == "__main__":
    grid_size = 64
    simulation_temp = 300.0  # Simulation temperature
    catalytic_threshold = 5.0 # Energy needed to tunnel

    # --- Initialize two parallel simulations ---
    params_light = GQR_Parameters(temperature=simulation_temp, substrate_isotope='light')
    params_heavy = GQR_Parameters(temperature=simulation_temp, substrate_isotope='heavy')

    V_gpu = initialize_3d_system(grid_size)

    soliton_energy_light = 0.1
    soliton_energy_heavy = 0.1

    psi_gpu_light = create_wave_packet(grid_size, soliton_energy_light)
    psi_gpu_heavy = create_wave_packet(grid_size, soliton_energy_heavy)

    # --- Simulation Loop and Static Plotting ---
    num_frames_to_run = 5000 # Number of simulation steps
    plot_interval = 100 # Plot every 100 frames

    print(f"Starting side-by-side GQR mechanism simulation for {num_frames_to_run} frames and saving plots...")

    for frame in range(num_frames_to_run):
        # --- GQR Ratchet Logic ---
        # Every 10 frames, attempt a ratchet "click"
        if frame % 10 == 0:
            # Light Isotope
            if np.random.rand() < params_light.p_confirm and soliton_energy_light < catalytic_threshold:
                soliton_energy_light += params_light.energy_increment
                psi_gpu_light = create_wave_packet(grid_size, soliton_energy_light) # Re-energize packet

            # Heavy Isotope
            if np.random.rand() < params_heavy.p_confirm and soliton_energy_heavy < catalytic_threshold:
                soliton_energy_heavy += params_heavy.energy_increment
                psi_gpu_heavy = create_wave_packet(grid_size, soliton_energy_heavy)

        # Evolve both wavefunctions for this frame
        psi_gpu_light = update_wavefunction_gpu(psi_gpu_light, V_gpu)
        psi_gpu_heavy = update_wavefunction_gpu(psi_gpu_heavy, V_gpu)

        # --- Plotting and Saving at Intervals ---
        if frame % plot_interval == 0 or frame == num_frames_to_run - 1:
            fig, axes = plt.subplots(1, 2, figsize=(20, 8))
            fig.patch.set_facecolor('black') # Set figure background to black
            fig.suptitle(f'GQR Mechanism Simulation at T = {simulation_temp}K - Frame {frame}', fontsize=20, fontweight='bold', color='white') # Set title color to white

            # --- Left Panel: Light Isotope ---
            ax_light = axes[0]
            ax_light.set_facecolor('black') # Set axes background to black
            V_cpu_slice = cp.asnumpy(V_gpu[:, grid_size // 2, :])
            ax_light.imshow(V_cpu_slice.T, cmap='viridis', origin='lower', extent=[0, grid_size, 0, grid_size], alpha=0.3)
            prob_density_light = np.abs(cp.asnumpy(psi_gpu_light[:, grid_size // 2, :]))**2
            img_light = ax_light.imshow(prob_density_light.T, cmap='inferno', origin='lower', extent=[0, grid_size, 0, grid_size], vmin=0, vmax=0.005) # Changed cmap to 'inferno' for black to red/yellow
            ax_light.set_title("Light Isotope (H)", fontsize=16, color='white') # Set title color to white
            ax_light.set_xlabel("X-axis", fontsize=12, color='white') # Set label color to white
            ax_light.set_ylabel("Z-axis", fontsize=12, color='white') # Set label color to white
            ax_light.tick_params(axis='x', colors='white') # Set tick color to white
            ax_light.tick_params(axis='y', colors='white') # Set tick color to white
            energy_text_light = ax_light.text(0.05, 0.95, '', transform=ax_light.transAxes, color='white', fontsize=14, va='top', bbox=dict(boxstyle='round,pad=0.5', fc='black', alpha=0.5))
            energy_text_light.set_text(f'Soliton Energy: {soliton_energy_light:.2f}\nStatus: {"TUNNELING" if soliton_energy_light >= catalytic_threshold else "Charging..."}')


            # --- Right Panel: Heavy Isotope ---
            ax_heavy = axes[1]
            ax_heavy.set_facecolor('black') # Set axes background to black
            ax_heavy.imshow(V_cpu_slice.T, cmap='viridis', origin='lower', extent=[0, grid_size, 0, grid_size], alpha=0.3)
            prob_density_heavy = np.abs(cp.asnumpy(psi_gpu_heavy[:, grid_size // 2, :]))**2
            img_heavy = ax_heavy.imshow(prob_density_heavy.T, cmap='inferno', origin='lower', extent=[0, grid_size, 0, grid_size], vmin=0, vmax=0.005) # Changed cmap to 'inferno' for black to red/yellow
            ax_heavy.set_title("Heavy Isotope (D)", fontsize=16, color='white') # Set title color to white
            ax_heavy.set_xlabel("X-axis", fontsize=12, color='white') # Set label color to white
            ax_heavy.tick_params(axis='x', colors='white') # Set tick color to white
            ax_heavy.tick_params(axis='y', colors='white') # Set tick color to white
            energy_text_heavy = ax_heavy.text(0.05, 0.95, '', transform=ax_heavy.transAxes, color='white', fontsize=14, va='top', bbox=dict(boxstyle='round,pad=0.5', fc='black', alpha=0.5))
            energy_text_heavy.set_text(f'Soliton Energy: {soliton_energy_heavy:.2f}\nStatus: {"TUNNELING" if soliton_energy_heavy >= catalytic_threshold else "Charging..."}')


            plt.setp(axes, yticks=[]) # Remove yticks from the right panel for cleaner side-by-side view
            plt.savefig(f'frame_{frame:04d}.png') # Save the figure
            plt.close(fig) # Close the figure to free memory


    print("Simulation and plotting complete.")

Starting side-by-side GQR mechanism simulation for 5000 frames and saving plots...


/tmp/ipython-input-243773178.py:121: RuntimeWarning: overflow encountered in square
  prob_density_light = np.abs(cp.asnumpy(psi_gpu_light[:, grid_size // 2, :]))**2
/usr/local/lib/python3.12/dist-packages/matplotlib/colors.py:2294: RuntimeWarning: overflow encountered in divide
  resdat /= (vmax - vmin)
/usr/local/lib/python3.12/dist-packages/matplotlib/colors.py:777: RuntimeWarning: overflow encountered in multiply
  xa *= self.N
/tmp/ipython-input-243773178.py:136: RuntimeWarning: overflow encountered in square
  prob_density_heavy = np.abs(cp.asnumpy(psi_gpu_heavy[:, grid_size // 2, :]))**2


Simulation and plotting complete.
